In [1]:
import pickle5 as pickle
import statistics
import math

In [2]:
# get required information from user
DataPath = "../" # the folder containing the required files
OptFN = "{}options_full.ini".format(DataPath)
NPos = 4 # number of positions of interest / partner

In [3]:
#### FUNCTIONS ####
# get run parameters from options file	
def get_Options(OptFN, OptionsDic, AllowedPars):
    try:
        optionsfile=open(OptFN, "r")
        for line in optionsfile :
            if line[:1] != "#" and len(line.strip("\n".strip(""))) > 1 :
                splitted_line = line.split(":")
                key=splitted_line[0].split(" ")[0]
                value=splitted_line[1].strip("\n").split(" ")[0]
                if key in AllowedPars :
                    OptionsDic[key] = value 
    except:
        print('Error reading %s file !' % (OptFN))

    return OptionsDic

#retrieve list of lists based on 2 separators
def double_split(Text, sep1, sep2):
    # further parse options
    WholeL = []
    MainL = Text.split(sep1)
    for text in MainL:
        WholeL.append(text.strip().split(sep2))

    return WholeL

In [4]:
## load required data
# the options
OptionsDic = {}
AllowedPars = ["Partners"]
OptionsDic = get_Options(OptFN, OptionsDic, AllowedPars)
ComplexesL = double_split(OptionsDic["Partners"], ";", "~")[0]
# the AAvarsDic
AAvarsDic = {} 
GlobalAAvarsDic = {}
with open(r"{}SR_AAvars_{}_vs_{}.pkl".format(DataPath,ComplexesL[0], ComplexesL[1]), "rb") as input_file:
    GlobalAAvarsDic = pickle.load(input_file)
    
## Drop irrelevant information
for complex in GlobalAAvarsDic.keys():
    AAvarsDic = GlobalAAvarsDic[complex]
    for key in AAvarsDic.keys():
        del AAvarsDic[key]['IndBCData']
    
    print("{} single mutant combinations found in SR_AAvars dictionary for complex partners {} (A) and {} (B) !".format(len(AAvarsDic.keys()), ComplexesL[0], ComplexesL[1]))

10956 single mutant combinations found in SR_AAvars dictionary for complex partners VN1551_VHH2 (A) and VN1554_TNFa-2 (B) !


In [5]:
# Peturbation analyis
#check if one of the partners have constant sequence
pA_set = set()
pB_set = set()
ConstSeqL = []
for key in AAvarsDic.keys():
    split_key = key.split(":")
    pA_set.add(split_key[0])
    pB_set.add(split_key[1])
if len(pA_set) == 1:
    ConstSeqL.append(ComplexesL[0])
if len(pB_set) == 1:   
    ConstSeqL.append(ComplexesL[1])

# get WT:WT data
WT_NormEnrich = AAvarsDic['WT:WT']['Stats'][0]
WT_SD = AAvarsDic['WT:WT']['Stats'][1]
WT_NdBCs = AAvarsDic['WT:WT']['Stats'][2]
WT_SE = WT_SD / math.sqrt(WT_NdBCs)
print("WT:WT data\n\tNormalized enrichment({:.1f}), sd({:.3f}), se({:.3f}) and number of double barcodes({})".format(WT_NormEnrich, WT_SD ,WT_SE, WT_NdBCs))

# get mutant positions perturbation score
PertDic = {}
NormEnrich = SD = SE = 0
for key in AAvarsDic.keys():
    MutPartner = None
    Pos = 0
    if key != "WT:WT" :
        split_key = key.split(":")
        if split_key[0] == "WT":
            MutPartner = ComplexesL[1] # Partner B
        if split_key[1] == "WT":
            MutPartner = ComplexesL[0] # Partner A
        if  MutPartner != None :   
            NormEnrich = AAvarsDic[key]['Stats'][0]
            SD = AAvarsDic[key]['Stats'][1]
            NdBCs = AAvarsDic[key]['Stats'][2]
            if NdBCs > 1:
                SE = SD / math.sqrt( WT_NdBCs )
                PertScore = (WT_NormEnrich - SD) - NormEnrich
                if MutPartner == ComplexesL[0]: # Partner A
                    Pos = int(split_key[0][1:-1])
                if MutPartner == ComplexesL[1]:  # Partner B
                    Pos = int(split_key[1][1:-1])
                if MutPartner in PertDic.keys():
                    if Pos in PertDic[MutPartner].keys():
                        PertDic[MutPartner][Pos][0].append(PertScore) 
                    else: # 1st observation of the position for a given mutation partner
                        PertDic[MutPartner][Pos] = [[PertScore], None, None, None, None]
                else: # 1st observation of the mutation partner
                    # PertDic["MutPartner"] {"Pos" : [Lis of perturbations, Mean, SD, SE, N] }
                    PertDic[MutPartner] = { Pos : [[PertScore], None, None, None, None] }  

for MutPartner in PertDic.keys():
    for Pos in PertDic[MutPartner].keys():
        N = len(PertDic[MutPartner][Pos][0])
        try:
            Average = statistics.mean(PertDic[MutPartner][Pos][0])
            SD = statistics.stdev(PertDic[MutPartner][Pos][0])
            SE = SD / math.sqrt(N)
        except:
            Average = PertDic[MutPartner][Pos][0][0]
            SD = None
            SE = None
        PertDic[MutPartner][Pos][1] = Average
        PertDic[MutPartner][Pos][2] = SD
        PertDic[MutPartner][Pos][3] = SE
        PertDic[MutPartner][Pos][4] = N

# print general information and reorder data
print("\nMutants data")
for partner in PertDic.keys():
    if partner not in ConstSeqL:
        print("\t{} positions found for {}".format( len(PertDic[partner]), partner) )
        PertDic[partner] = sorted(PertDic[partner].items(), key=lambda item: item[1][1], reverse=True)

WT:WT data
	Normalized enrichment(1.0), sd(0.281), se(0.007) and number of double barcodes(1450)

Mutants data
	15 positions found for VN1551_VHH2
	15 positions found for VN1554_TNFa-2


In [6]:
# print position perturbation rank (epitopes)
for MutPartner in PertDic.keys():
    print ("{} most perturbed positions ranking: ".format(MutPartner))
    for i in range(NPos):
        dataL = list(PertDic[MutPartner][i])
        print("\tPosition {} has a mean pertubation score against WT partner = {:.3f} ({} mutants)".format(dataL[0], dataL[1][1], dataL[1][-1]))

VN1551_VHH2 most perturbed positions ranking: 
	Position 33 has a mean pertubation score against WT partner = 0.496 (10 mutants)
	Position 57 has a mean pertubation score against WT partner = 0.489 (12 mutants)
	Position 35 has a mean pertubation score against WT partner = 0.463 (12 mutants)
	Position 54 has a mean pertubation score against WT partner = 0.349 (9 mutants)
VN1554_TNFa-2 most perturbed positions ranking: 
	Position 137 has a mean pertubation score against WT partner = 0.767 (4 mutants)
	Position 90 has a mean pertubation score against WT partner = 0.681 (10 mutants)
	Position 79 has a mean pertubation score against WT partner = 0.580 (4 mutants)
	Position 135 has a mean pertubation score against WT partner = 0.541 (10 mutants)
